Final Deliverable

Title

Fall 2024 Data Science Project

Your name(s)

Contributions:

For each member, list which of the following sections they worked on, and
summarize the contributions in 1-2 sentences. Be specific!

A: Project idea

B: Dataset Curation and Preprocessing

C: Data Exploration and Summary Statistics

D: ML Algorithm Design/Development

E: ML Algorithm Training and Test Data Analysis

F: Visualization, Result Analysis, Conclusion

G: Final Tutorial Report Creation

H: Additional (not listed above)

**Introduction**

The introduction should motivate your work: what is your topic? What
question(s) are you trying to answer with your analysis? Why is answering those
questions important?


**Data curation**


ite the source(s) of your data. Explain what it is. Transform the data
so that it is ready for analysis. For example, set up a database and use SQL to query
for data, or organize a pandas DataFrame

**Exploratory data analysis**

**CHECKPOINT TWO**

**Primary analysis**

Based on the results of your exploration, choose a machine
learning technique (e.g., classification, regression, clustering, etc.) that will help you
answer the questions you posed in the introduction. Explain your reasoning

**Visualization**

Explain the results and insights of your primary analysis with at least
one plot. Make sure that every element of the plots are labeled and explained (don’t
forget to include a legend!).

**Insights and Conclusions**


After reading through the project, does an uninformed
reader feel informed about the topic? Would a reader who already knew about the
topic feel like they learned more about it?